In [ ]:
# TODO fix duplicates
import pandas as pd
from pandas import json_normalize
from datetime import date
pd.set_option('display.max_columns', None)

In [2]:
df_work_raw = catalog.load('raw/openalex/work#parquet')

                    INFO     Loading data from raw/openalex/work#parquet (ParquetDataset)...    ]8;id=197730;file:///home/pablo/dev/SEDICI/dw/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=739364;file:///home/pablo/dev/SEDICI/dw/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#482\482]8;;\

In [3]:
df_work_raw

,id,doi,title,display_name,publication_year,publication_date,ids,language,primary_location,type,type_crossref,indexed_in,open_access,authorships,institution_assertions,countries_distinct_count,institutions_distinct_count,corresponding_author_ids,corresponding_institution_ids,apc_list,apc_paid,fwci,is_authors_truncated,has_fulltext,fulltext_origin,cited_by_count,citation_normalized_percentile,cited_by_percentile_year,biblio,is_retracted,is_paratext,primary_topic,topics,keywords,concepts,mesh,locations_count,locations,best_oa_location,sustainable_development_goals,grants,datasets,versions,referenced_works_count,referenced_works,related_works,abstract_inverted_index,cited_by_api_url,counts_by_year,updated_date,created_date
0,https://openalex.org/W4211046286,https://doi.org/10.1088/1748-0221/3/08/s08003,The ATLAS Experiment at the CERN Large Hadron ...,The ATLAS Experiment at the CERN Large Hadron ...,2008,2008-08-14,{'doi': 'https://doi.org/10.1088/1748-0221/3/0...,en,"{'is_accepted': True, 'is_oa': True, 'is_publi...",article,journal-article,[crossref],"{'any_repository_has_fulltext': True, 'is_oa':...",[{'affiliations': [{'institution_ids': array([...,[],36,187,[],[],None,None,54.587,True,True,pdf,3507,"{'is_in_top_10_percent': True, 'is_in_top_1_pe...","{'max': 100, 'min': 99}","{'first_page': 'S08003', 'issue': '08', 'last_...",False,False,{'display_name': 'Advancements in Particle Det...,[{'display_name': 'Advancements in Particle De...,"[{'display_name': 'Atlas detector', 'id': 'htt...","[{'display_name': 'Large Hadron Collider', 'id...",[],8,"[{'is_accepted': True, 'is_oa': True, 'is_publ...","{'is_accepted': True, 'is_oa': True, 'is_publi...",[],[],[],[],153,"[https://openalex.org/W1578925023, https://ope...","[https://openalex.org/W4385358068, https://ope...","{'""ankle""': None, '""ridge.""': None, '""smart""':...",https://api.openalex.org/works?filter=cites:W4...,"[{'cited_by_count': 201, 'year': 2024}, {'cite...",2024-10-13T11:28:57.211497,2022-02-13
1,https://openalex.org/W4292403095,https://doi.org/10.1051/0004-6361:20041864,The Leiden/Argentine/Bonn (LAB) Survey of Gala...,The Leiden/Argentine/Bonn (LAB) Survey of Gala...,2005,2005-08-29,{'doi': 'https://doi.org/10.1051/0004-6361:200...,en,"{'is_accepted': True, 'is_oa': True, 'is_publi...",article,journal-article,"[arxiv, crossref, datacite]","{'any_repository_has_fulltext': True, 'is_oa':...",[{'affiliations': [{'institution_ids': array([...,[],3,3,[],[],None,None,22.311,None,True,pdf,3151,"{'is_in_top_10_percent': True, 'is_in_top_1_pe...","{'max': 100, 'min': 99}","{'first_page': '775', 'issue': '2', 'last_page...",False,False,{'display_name': 'High-Energy Astrophysics and...,[{'display_name': 'High-Energy Astrophysics an...,"[{'display_name': 'Relativistic Jets', 'id': '...","[{'display_name': 'Physics', 'id': 'https://op...",[],7,"[{'is_accepted': True, 'is_oa': True, 'is_publ...","{'is_accepted': True, 'is_oa': True, 'is_publi...",[],[],[],[https://openalex.org/W4292403095],6,"[https://openalex.org/W1539512060, https://ope...","[https://openalex.org/W4296758524, https://ope...","{'""ankle""': None, '""ridge.""': None, '""smart""':...",https://api.openalex.org/works?filter=cites:W4...,"[{'cited_by_count': 58, 'year': 2024}, {'cited...",2024-10-05T06:53:59.800641,2022-08-20
2,https://openalex.org/W2098406858,None,The ATLAS Experiment at the CERN Large Hadron ...,The ATLAS Experiment at the CERN Large Hadron ...,2020,2020-02-23,"{'doi': None, 'mag': '2098406858', 'openalex':...",en,"{'is_accepted': False, 'is_oa': False, 'is_pub...",book,book,[],"{'any_repository_has_fulltext': False, 'is_oa'...",[{'affiliations': [{'institution_ids': array([...,[],32,170,[],[],None,None,8.066,True,False,None,2473,"{'is_in_top_10_percent': False, 'is_in_top_1_p...","{'max': 100, 'min': 99}","{'first_page': None, 'issue': None, 'last_page...",False,False,{'display_name': 'Particle Physics and High-En...,[{'display_name': 'Particle Physics and High-E...,"[{'display_name': 'Atlas 

In [ ]:
def land_researchproduct_openaire(df: pd.DataFrame)-> pd.DataFrame:
    df = df.convert_dtypes()
    return df

    def get_value_by_attr(value, attr, attr_filter, attr_value):
        if isinstance(value, dict):
            if value.get(attr_filter) == attr_value:
                return value.get(attr)
        elif isinstance(value, list):
            for item in value:
                if item.get(attr_filter) == attr_value:
                    return item.get(attr)
        return None

    def convert_to_dict_or_list(value):
        try:
            return ast.literal_eval(value)
        except (ValueError, SyntaxError):
            return value

    df_researchproduct = pd.DataFrame()

    df_researchproduct['id'] = df['dri:objIdentifier']
    df_researchproduct['dateOfCollection'] = df['dri:dateOfCollection']

    df['collectedfrom'] = df['collectedfrom'].apply(convert_to_dict_or_list)
    df_researchproduct_collectedfrom = df.loc[:,['dri:objIdentifier','collectedfrom']].explode('collectedfrom').reset_index(drop=True)
    df_researchproduct_collectedfrom['collectedfrom'] = df_researchproduct_collectedfrom['collectedfrom']

    df['originalId'] = df['originalId'].apply(convert_to_dict_or_list)
    df_researchproduct_originalId = df.loc[:,['dri:objIdentifier','originalId']].explode('originalId').reset_index(drop=True)

    df['pid'] = df['pid'].apply(convert_to_dict_or_list)

    df_researchproduct['doi'] = df.loc[:,'pid'].apply(get_value_by_attr, attr='#text', attr_filter='@classid' ,attr_value='doi')
    df_researchproduct['pmid'] = df.loc[:,'pid'].apply(get_value_by_attr, attr='#text', attr_filter='@classid' ,attr_value='pmid')
    df_researchproduct['pmc'] = df.loc[:,'pid'].apply(get_value_by_attr, attr='#text', attr_filter='@classid' ,attr_value='pmc')
    df_researchproduct['arXiv'] = df.loc[:,'pid'].apply(get_value_by_attr, attr='#text', attr_filter='@classid' ,attr_value='arXiv')
    df_researchproduct['uniprot'] = df.loc[:,'pid'].apply(get_value_by_attr, attr='#text', attr_filter='@classid' ,attr_value='uniprot')
    df_researchproduct['ena'] = df.loc[:,'pid'].apply(get_value_by_attr, attr='#text', attr_filter='@classid' ,attr_value='ena')
    df_researchproduct['pdb'] = df.loc[:,'pid'].apply(get_value_by_attr, attr='#text', attr_filter='@classid' ,attr_value='pdb')

    df['measure'] = df['measure'].apply(convert_to_dict_or_list)
    df_researchproduct_measure = df.loc[:,['dri:objIdentifier','measure']].explode('measure').reset_index(drop=True)

    df['title'] = df['title'].apply(convert_to_dict_or_list)
    df_researchproduct_title = df.loc[:,['dri:objIdentifier','title']].explode('title').reset_index(drop=True)

    df['bestaccessright'] = df['bestaccessright'].apply(convert_to_dict_or_list)
    df_researchproduct['bestaccessright'] = df.loc[:,'bestaccessright'].apply(lambda x: x.get('@classname'))

    df_researchproduct['dateofacceptance'] = df['dateofacceptance']
    
    df_researchproduct['publisher'] = df.loc[:,'publisher']
    
    df['source'] = df['source'].apply(convert_to_dict_or_list)
    df_researchproduct_source = df.loc[:,['dri:objIdentifier','source']].explode('source').reset_index(drop=True)

    df['resulttype'] = df['resulttype'].apply(convert_to_dict_or_list)
    df_researchproduct['resulttype'] = df['resulttype'].apply(lambda x: x.get('@classname'))

    df['resourcetype'] = df['resourcetype'].apply(convert_to_dict_or_list)
    df_researchproduct['resourcetype'] = df['resourcetype'].apply(lambda x: x.get('@classname'))
    
    df_researchproduct['isgreen'] = df['isgreen']

    df_researchproduct['openaccesscolor'] = df['openaccesscolor']

    df_researchproduct['isindiamondjournal'] = df['isindiamondjournal']
    
    df_researchproduct['publiclyfunded'] = df['publiclyfunded']
    # df['journal'] = df['journal'].apply(convert_to_dict_or_list)
    # df_researchproduct['journal'] = df['journal']

    df['subject'] = df['subject'].apply(convert_to_dict_or_list)
    df_researchproduct_subject = df.loc[:,['dri:objIdentifier','subject']].explode('subject').reset_index(drop=True)

    df_researchproduct['load_datetime'] = date.today()

    return df_researchproduct, df_researchproduct_originalId, df_researchproduct_source


In [5]:
df_work_duplicated = pd.DataFrame()
df_work, df_work_duplicated = land_work_openalex(df_work_raw)

[10/15/24 09:18:11] WARNING  /tmp/ipykernel_54445/1777742255.py:82: FutureWarning: The default      ]8;id=843435;file:///usr/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=688024;file:///usr/lib/python3.10/warnings.py#109\109]8;;\
                             value of regex will change from True to False in a future version. In                 
                             addition, single character regular expressions will *not* be treated                  
                             as literal strings when regex=True.                                                   
                               df_work.columns = df_work.columns.str.replace('.', '_')                             
                                                                                                                   

duplicated ids:
 34     https://openalex.org/W3122013459
56     https://openalex.org/W2033025494
123    https://openalex.org/W2754962649
132    https://openalex.org/W2165825148
Name: work_id, dtype: string


In [6]:
df_work

,work_id,doi,title,display_name,publication_year,publication_date,language,type,type_crossref,countries_distinct_count,institutions_distinct_count,fwci,is_authors_truncated,has_fulltext,fulltext_origin,cited_by_count,is_retracted,is_paratext,locations_count,referenced_works_count,cited_by_api_url,updated_date,created_date,mag,pmcid,pmid,biblio_first_page,biblio_issue,biblio_last_page,biblio_volume,any_repository_has_fulltext,is_oa,oa_status,oa_url,cited_by_percentile_year_max,cited_by_percentile_year_min,citation_normalized_percentile_is_in_top_10_percent,citation_normalized_percentile_is_in_top_1_percent,citation_normalized_percentile_value,apc_list_currency,apc_list_provenance,apc_list_value,apc_list_value_usd,apc_paid_currency,apc_paid_provenance,apc_paid_value,apc_paid_value_usd,indexed_in_arxiv,indexed_in_doaj,indexed_in_crossref,indexed_in_pubmed,load_datetime
0,https://openalex.org/W4211046286,10.1088/1748-0221/3/08/s08003,The ATLAS Experiment at the CERN Large Hadron ...,The ATLAS Experiment at the CERN Large Hadron ...,2008,2008-08-14,en,article,journal-article,36,187,54.587,True,True,pdf,3507,False,False,8,153,https://api.openalex.org/works?filter=cites:W4...,2024-10-13T11:28:57.211497,2022-02-13,NO DATA,NO DATA,NO DATA,S08003,08,S08003,3,True,True,bronze,https://iopscience.iop.org/article/10.1088/174...,100,99,True,True,0.999906,NO DATA,NO DATA,0,0,NO DATA,NO DATA,0,0,False,False,True,False,2024-10-15
1,https://openalex.org/W4292403095,10.1051/0004-6361:20041864,The Leiden/Argentine/Bonn (LAB) Survey of Gala...,The Leiden/Argentine/Bonn (LAB) Survey of Gala...,2005,2005-08-29,en,article,journal-article,3,3,22.311,<NA>,True,pdf,3151,False,False,7,6,https://api.openalex.org/works?filter=cites:W4...,2024-10-05T06:53:59.800641,2022-08-20,NO DATA,NO DATA,NO DATA,775,2,782,440,True,True,bronze,https://www.aanda.org/articles/aa/pdf/2005/35/...,100,99,True,True,0.999901,NO DATA,NO DATA,0,0,NO DATA,NO DATA,0,0,True,False,True,False,2024-10-15
2,https://openalex.org/W2098406858,<NA>,The ATLAS Experiment at the CERN Large Hadron ...,The ATLAS Experiment at the CERN Large Hadron ...,2020,2020-02-23,en,book,book,32,170,8.066,True,False,<NA>,2473,False,False,1,220,https://api.openalex.org/works?filter=cites:W2...,2024-10-08T14:17:10.855397,2016-06-24,2098406858,NO DATA,NO DATA,NO DATA,NO DATA,NO DATA,NO DATA,False,False,closed,NO DATA,100,99,False,False,0.575758,NO DATA,NO DATA,0,0,NO DATA,NO DATA,0,0,False,False,False,False,2024-10-15
3,https://openalex.org/W2501864044,10.1093/mnras/stx721,The clustering of galaxies in the completed SD...,The clustering of galaxies in the completed SD...,2017,2017-03-24,en,article,journal-article,12,57,114.247,<NA>,True,pdf,2397,False,False,21,234,https://api.openalex.org/works?filter=cites:W2...,2024-10-13T11:36:31.218894,2016-08-23,2501864044,NO DATA,NO DATA,2617,3,2652,470,True,True,bronze,https://academic.oup.com/mnras/article-pdf/470...,100,99,True,True,0.99995,GBP,doaj,2310,2833,NO DATA,NO DATA,0,0,True,False,True,False,2024-10-15
4,https://openalex.org/W2012331775,10.1038/nature10158,Genome sequence and analysis of the tuber crop...,Genome sequence and analysis of the tuber crop...,2011,2011-07-01,en,article,journal-article,12,19,147.958,<NA>,True,ngrams,1937,False,False,5,55,https://api.openalex.org/works?filter=cites:W2...,2024-10-08T17:20:44.493312,2016-06-24,2012331775,NO DATA,https://pubmed.ncbi.nlm.nih.gov/21743474,189,7355,195,475,True,True,hybrid,https://www.nature.com/articles/nature10158.pdf,100,99,True,True,0.999978,EUR,doaj,9750,11690,EUR,doaj,9750,11690,False,False,True,True,2024-10-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,https://openalex.org/W2972599699,10.1140/epjc/s10052-019-7594-6,Search for electroweak production of charginos...,Search for electroweak production of charginos...,2020,2020-02-01,e

In [7]:
df_work_duplicated

,work_id,doi,title,display_name,publication_year,publication_date,language,type,type_crossref,countries_distinct_count,institutions_distinct_count,fwci,is_authors_truncated,has_fulltext,fulltext_origin,cited_by_count,is_retracted,is_paratext,locations_count,referenced_works_count,cited_by_api_url,updated_date,created_date,mag,pmcid,pmid,biblio_first_page,biblio_issue,biblio_last_page,biblio_volume,any_repository_has_fulltext,is_oa,oa_status,oa_url,cited_by_percentile_year_max,cited_by_percentile_year_min,citation_normalized_percentile_is_in_top_10_percent,citation_normalized_percentile_is_in_top_1_percent,citation_normalized_percentile_value,apc_list_currency,apc_list_provenance,apc_list_value,apc_list_value_usd,apc_paid_currency,apc_paid_provenance,apc_paid_value,apc_paid_value_usd,indexed_in_arxiv,indexed_in_doaj,indexed_in_crossref,indexed_in_pubmed,load_datetime
29,https://openalex.org/W3122013459,10.1016/j.jpubeco.2012.10.009,Biased perceptions of income distribution and ...,Biased perceptions of income distribution and ...,2012,2012-11-12,en,article,journal-article,3,4,42.262,<NA>,True,pdf,590,False,False,7,62,https://api.openalex.org/works?filter=cites:W3...,2024-10-09T18:30:47.502511,2021-02-01,3122013459,NO DATA,NO DATA,100,NO DATA,112,98,True,True,green,https://ri.conicet.gov.ar/bitstream/11336/7675...,100,99,True,True,0.999888,USD,doaj,3650,3650,NO DATA,NO DATA,0,0,False,False,True,False,2024-10-15
34,https://openalex.org/W3122013459,10.1016/j.jpubeco.2012.10.009,Biased perceptions of income distribution and ...,Biased perceptions of income distribution and ...,2013,2013-02-01,en,article,journal-article,3,4,42.262,<NA>,True,pdf,562,False,False,7,62,https://api.openalex.org/works?filter=cites:W3...,2024-09-15T04:20:12.452442,2021-02-01,3122013459,NO DATA,NO DATA,100,NO DATA,112,98,True,True,green,https://ri.conicet.gov.ar/bitstream/11336/7675...,100,99,True,True,0.999888,USD,doaj,3650,3650,NO DATA,NO DATA,0,0,False,False,True,False,2024-10-15
54,https://openalex.org/W2033025494,10.1016/j.actbio.2009.04.041,Corrosion behaviour of AZ31 magnesium alloy wi...,Corrosion behaviour of AZ31 magnesium alloy wi...,2009,2009-05-14,en,article,journal-article,2,5,14.717,<NA>,True,pdf,451,False,False,3,34,https://api.openalex.org/works?filter=cites:W2...,2024-10-09T12:35:21.037615,2016-06-24,2033025494,NO DATA,https://pubmed.ncbi.nlm.nih.gov/19446048,1763,5,1771,6,True,True,green,https://digital.csic.es/bitstream/10261/59353/...,100,99,True,True,0.999866,USD,doaj,3210,3210,NO DATA,NO DATA,0,0,False,False,True,True,2024-10-15
56,https://openalex.org/W2033025494,10.1016/j.actbio.2009.04.041,Corrosion behaviour of AZ31 magnesium alloy wi...,Corrosion behaviour of AZ31 magnesium alloy wi...,2010,2010-05-01,en,article,journal-article,2,5,14.717,<NA>,True,pdf,448,False,False,3,34,https://api.openalex.org/works?filter=cites:W2...,2024-09-14T19:23:28.108680,2016-06-24,2033025494,NO DATA,https://pubmed.ncbi.nlm.nih.gov/19446048,1763,5,1771,6,True,True,green,https://digital.csic.es/bitstream/10261/59353/...,100,99,True,True,0.999866,USD,doaj,3210,3210,NO DATA,NO DATA,0,0,False,False,True,True,2024-10-15
122,https://openalex.org/W2754962649,10.1016/j.jiec.2017.09.012,Rollinia occidentalis extract as green corrosi...,Rollinia occidentalis extract as green corrosi...,2017,2017-09-13,en,article,journal-article,1,5,15.963,<NA>,True,ngrams,293,False,False,1,28,https://api.openalex.org/works?filter=cites:W2...,2024-10-10T00:33:14.158311,2017-09-25,2754962649,NO DATA,NO DATA,92,NO DATA,99,58,False,False,closed,NO DATA,100,99,True,True,0.999975,USD,doaj,2500,2500,NO DATA,NO DATA,0,0,False,False,True,False,2024-10-15
123,https://openalex.org/W2754962649,10.1016/j.jiec.2017.09.012,Rollinia occidentalis extract as green corrosi...,Rollinia occidentalis extract as green corrosi...,2018,2018-02-01,en,article,journal-article,1,5,15.963,<NA>,True,ngrams,290,False,False,1,28,https://api.openalex.org/works?filter=cites:W2...,2024-09-15T13:58:40.846136,2017-0

Verificamos si hay duplicados

In [8]:
filter = df_work['work_id'].duplicated()
duplicated_id = df_work[filter]['work_id']
print('duplicated ids:\n',duplicated_id)

duplicated ids:
 Series([], Name: work_id, dtype: string)


In [9]:
duplicated_filter = df_work['work_id'].isin(duplicated_id)
df_work[duplicated_filter]


,work_id,doi,title,display_name,publication_year,publication_date,language,type,type_crossref,countries_distinct_count,institutions_distinct_count,fwci,is_authors_truncated,has_fulltext,fulltext_origin,cited_by_count,is_retracted,is_paratext,locations_count,referenced_works_count,cited_by_api_url,updated_date,created_date,mag,pmcid,pmid,biblio_first_page,biblio_issue,biblio_last_page,biblio_volume,any_repository_has_fulltext,is_oa,oa_status,oa_url,cited_by_percentile_year_max,cited_by_percentile_year_min,citation_normalized_percentile_is_in_top_10_percent,citation_normalized_percentile_is_in_top_1_percent,citation_normalized_percentile_value,apc_list_currency,apc_list_provenance,apc_list_value,apc_list_value_usd,apc_paid_currency,apc_paid_provenance,apc_paid_value,apc_paid_value_usd,indexed_in_arxiv,indexed_in_doaj,indexed_in_crossref,indexed_in_pubmed,load_datetime


In [10]:
df_work[~duplicated_filter]


,work_id,doi,title,display_name,publication_year,publication_date,language,type,type_crossref,countries_distinct_count,institutions_distinct_count,fwci,is_authors_truncated,has_fulltext,fulltext_origin,cited_by_count,is_retracted,is_paratext,locations_count,referenced_works_count,cited_by_api_url,updated_date,created_date,mag,pmcid,pmid,biblio_first_page,biblio_issue,biblio_last_page,biblio_volume,any_repository_has_fulltext,is_oa,oa_status,oa_url,cited_by_percentile_year_max,cited_by_percentile_year_min,citation_normalized_percentile_is_in_top_10_percent,citation_normalized_percentile_is_in_top_1_percent,citation_normalized_percentile_value,apc_list_currency,apc_list_provenance,apc_list_value,apc_list_value_usd,apc_paid_currency,apc_paid_provenance,apc_paid_value,apc_paid_value_usd,indexed_in_arxiv,indexed_in_doaj,indexed_in_crossref,indexed_in_pubmed,load_datetime
0,https://openalex.org/W4211046286,10.1088/1748-0221/3/08/s08003,The ATLAS Experiment at the CERN Large Hadron ...,The ATLAS Experiment at the CERN Large Hadron ...,2008,2008-08-14,en,article,journal-article,36,187,54.587,True,True,pdf,3507,False,False,8,153,https://api.openalex.org/works?filter=cites:W4...,2024-10-13T11:28:57.211497,2022-02-13,NO DATA,NO DATA,NO DATA,S08003,08,S08003,3,True,True,bronze,https://iopscience.iop.org/article/10.1088/174...,100,99,True,True,0.999906,NO DATA,NO DATA,0,0,NO DATA,NO DATA,0,0,False,False,True,False,2024-10-15
1,https://openalex.org/W4292403095,10.1051/0004-6361:20041864,The Leiden/Argentine/Bonn (LAB) Survey of Gala...,The Leiden/Argentine/Bonn (LAB) Survey of Gala...,2005,2005-08-29,en,article,journal-article,3,3,22.311,<NA>,True,pdf,3151,False,False,7,6,https://api.openalex.org/works?filter=cites:W4...,2024-10-05T06:53:59.800641,2022-08-20,NO DATA,NO DATA,NO DATA,775,2,782,440,True,True,bronze,https://www.aanda.org/articles/aa/pdf/2005/35/...,100,99,True,True,0.999901,NO DATA,NO DATA,0,0,NO DATA,NO DATA,0,0,True,False,True,False,2024-10-15
2,https://openalex.org/W2098406858,<NA>,The ATLAS Experiment at the CERN Large Hadron ...,The ATLAS Experiment at the CERN Large Hadron ...,2020,2020-02-23,en,book,book,32,170,8.066,True,False,<NA>,2473,False,False,1,220,https://api.openalex.org/works?filter=cites:W2...,2024-10-08T14:17:10.855397,2016-06-24,2098406858,NO DATA,NO DATA,NO DATA,NO DATA,NO DATA,NO DATA,False,False,closed,NO DATA,100,99,False,False,0.575758,NO DATA,NO DATA,0,0,NO DATA,NO DATA,0,0,False,False,False,False,2024-10-15
3,https://openalex.org/W2501864044,10.1093/mnras/stx721,The clustering of galaxies in the completed SD...,The clustering of galaxies in the completed SD...,2017,2017-03-24,en,article,journal-article,12,57,114.247,<NA>,True,pdf,2397,False,False,21,234,https://api.openalex.org/works?filter=cites:W2...,2024-10-13T11:36:31.218894,2016-08-23,2501864044,NO DATA,NO DATA,2617,3,2652,470,True,True,bronze,https://academic.oup.com/mnras/article-pdf/470...,100,99,True,True,0.99995,GBP,doaj,2310,2833,NO DATA,NO DATA,0,0,True,False,True,False,2024-10-15
4,https://openalex.org/W2012331775,10.1038/nature10158,Genome sequence and analysis of the tuber crop...,Genome sequence and analysis of the tuber crop...,2011,2011-07-01,en,article,journal-article,12,19,147.958,<NA>,True,ngrams,1937,False,False,5,55,https://api.openalex.org/works?filter=cites:W2...,2024-10-08T17:20:44.493312,2016-06-24,2012331775,NO DATA,https://pubmed.ncbi.nlm.nih.gov/21743474,189,7355,195,475,True,True,hybrid,https://www.nature.com/articles/nature10158.pdf,100,99,True,True,0.999978,EUR,doaj,9750,11690,EUR,doaj,9750,11690,False,False,True,True,2024-10-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,https://openalex.org/W2972599699,10.1140/epjc/s10052-019-7594-6,Search for electroweak production of charginos...,Search for electroweak production of charginos...,2020,2020-02-01,e

## Ejemplo de work duplicado

In [11]:
filter = df_work_raw['id'] == 'https://openalex.org/W3122013459'
df_work_raw[filter]

,id,doi,title,display_name,publication_year,publication_date,ids,language,primary_location,type,type_crossref,indexed_in,open_access,authorships,institution_assertions,countries_distinct_count,institutions_distinct_count,corresponding_author_ids,corresponding_institution_ids,apc_list,apc_paid,fwci,is_authors_truncated,has_fulltext,fulltext_origin,cited_by_count,citation_normalized_percentile,cited_by_percentile_year,biblio,is_retracted,is_paratext,primary_topic,topics,keywords,concepts,mesh,locations_count,locations,best_oa_location,sustainable_development_goals,grants,datasets,versions,referenced_works_count,referenced_works,related_works,abstract_inverted_index,cited_by_api_url,counts_by_year,updated_date,created_date
4,https://openalex.org/W3122013459,https://doi.org/10.1016/j.jpubeco.2012.10.009,Biased perceptions of income distribution and ...,Biased perceptions of income distribution and ...,2012,2012-11-12,{'doi': 'https://doi.org/10.1016/j.jpubeco.201...,en,"{'is_accepted': False, 'is_oa': False, 'is_pub...",article,journal-article,[crossref],"{'any_repository_has_fulltext': True, 'is_oa':...",[{'affiliations': [{'institution_ids': array([...,[],3,4,[https://openalex.org/A5004025028],"[https://openalex.org/I874386039, https://open...","{'currency': 'USD', 'provenance': 'doaj', 'val...",None,42.262,None,True,pdf,590,"{'is_in_top_10_percent': True, 'is_in_top_1_pe...","{'max': 100, 'min': 99}","{'first_page': '100', 'issue': None, 'last_pag...",False,False,{'display_name': 'Gender Inequality and Labor ...,[{'display_name': 'Gender Inequality and Labor...,"[{'display_name': 'Redistribution (election)',...","[{'display_name': 'Redistribution (election)',...",[],7,"[{'is_accepted': False, 'is_oa': False, 'is_pu...","{'is_accepted': True, 'is_oa': True, 'is_publi...","[{'display_name': 'No poverty', 'id': 'https:/...","[{'award_id': None, 'funder': 'https://openale...",[],[],62,"[https://openalex.org/W1263955588, https://ope...","[https://openalex.org/W4387518190, https://ope...","{'""ankle""': None, '""ridge.""': None, '""smart""':...",https://api.openalex.org/works?filter=cites:W3...,"[{'cited_by_count': 59, 'year': 2024}, {'cited...",2024-10-09T18:30:47.502511,2021-02-01
9,https://openalex.org/W3122013459,https://doi.org/10.1016/j.jpubeco.2012.10.009,Biased perceptions of income distribution and ...,Biased perceptions of income distribution and ...,2013,2013-02-01,{'doi': 'https://doi.org/10.1016/j.jpubeco.201...,en,"{'is_accepted': False, 'is_oa': False, 'is_pub...",article,journal-article,[crossref],"{'any_repository_has_fulltext': True, 'is_oa':...",[{'affiliations': [{'institution_ids': array([...,[],3,4,[https://openalex.org/A5004025028],"[https://openalex.org/I874386039, https://open...","{'currency': 'USD', 'provenance': 'doaj', 'val...",None,42.262,None,True,pdf,562,"{'is_in_top_10_percent': True, 'is_in_top_1_pe...","{'max': 100, 'min': 99}","{'first_page': '100', 'issue': None, 'last_pag...",False,False,{'display_name': 'Gender Inequality and Labor ...,[{'display_name': 'Gender Inequality and Labor...,"[{'display_name': 'Redistribution (election)',...","[{'display_name': 'Redistribution (election)',...",[],7,"[{'is_accepted': False, 'is_oa': False, 'is_pu...","{'is_accepted': True, 'is_oa': True, 'is_publi...","[{'display_name': 'No poverty', 'id': 'https:/...","[{'award_id': None, 'funder': 'https://openale...",[],[],62,"[https://openalex.org/W1263955588, https://ope...","[https://openalex.org/W4387518190, https://ope...","{'""ankle""': None, '""ridge.""': None, '""smart""':...",https://api.openalex.org/works?filter=cites:W3...,"[{'cited_by_count': 59, 'year': 2024}, {'cited...",2024-09-15T04:20:12.452442,2021-02-01
